<a href="https://www.kaggle.com/code/ayushkhaire/crypto-dataset-daily-updates-top-1000?scriptVersionId=200497805" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json as js
import subprocess
import logging
import time
import shutil

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2018-07-19 13:30:00,TLRY,23.049999,22.389999,24.000000,20.100000,11934500,22.389999
1,2018-07-20 13:30:00,TLRY,24.250000,29.770000,31.799999,23.500000,13938200,29.770000
2,2018-07-23 13:30:00,TLRY,33.480000,29.450001,34.099998,29.309999,10042900,29.450001
3,2018-07-24 13:30:00,TLRY,28.799999,25.360001,29.430000,25.250000,5524200,25.360001
4,2018-07-25 13:30:00,TLRY,25.309999,26.490000,27.150000,24.200001,3844100,26.490000
...,...,...,...,...,...,...,...,...
35783,2024-10-04 13:30:00,SYM,23.700001,23.840000,24.139999,22.940001,1277500,23.840000
35784,2024-10-07 13:30:00,SYM,23.850000,23.719999,23.920000,23.270000,688000,23.719999
35785,2024-10-08 13:30:00,SYM,23.530001,23.379999,23.840000,22.959999,844300,23.379999
35786,2024-10-09 13:30:00,SYM,23.450001,23.240000,23.809999,23.150000,519200,23.240000


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = f'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'
link

'https://finance.yahoo.com/markets/crypto/all/?start=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(20)):
    url = f'https://finance.yahoo.com/markets/crypto/all/?start={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 20/20 [00:13<00:00,  1.44it/s]


In [6]:
symbol_list = []

for r in tqdm(resp_main_list):
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('span',{'class':'symbol'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

100%|██████████| 20/20 [00:09<00:00,  2.15it/s]

2520


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [7]:
'''

llink1 = https://query1.finance.yahoo.com/v8/finance/chart/GLM-USD?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1=1510185600&period2=1725897789&symbol=GLM-USD&userYfid=true&lang=en-US&region=US

here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))

period2 = current_timestamp  

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1420070400&period2=1728663393


# create stocks directory

In [8]:
os.mkdir('crypto_data')

# Download all files

In [9]:
def return_timestamp(unix_timestamps, date_format='%Y-%m-%d %H:%M:%S'):
    formatted_dates = []
    for ut in unix_timestamps:
        date = datetime.fromtimestamp(ut)
        formatted_date = date.strftime(date_format)
        formatted_dates.append(formatted_date)
    return formatted_dates

print(return_timestamp([1725898133]))

['2024-09-09 16:08:53']


In [10]:
for stock in tqdm(symbol_list):
    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{stock}?events=capitalGain%7Cdiv%7Csplit&formatted=true&includeAdjustedClose=true&interval=1d&period1={period1}&period2={period2}&symbol={stock}&userYfid=true&lang=en-US&region=US'
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.json", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.json")
    else:
#         print(f"Failed to download file : {stock}.json")
        pass
#     time.sleep(3)

100%|██████████| 2520/2520 [10:22<00:00,  4.05it/s]


In [11]:
symbol_list_dir = os.listdir('/kaggle/working/crypto_data/')
len(symbol_list_dir)

19

# Read a sample

In [12]:
# jsonf = pd.read_json(f'/kaggle/working/crypto_data/BTC-USD.json')
# jsondict = jsonf.to_dict()
# timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
# sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
# symbol = list(sym for s in range(len(timestamp)))
# low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
# high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
# volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
# open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
# close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
# adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
# dates = return_timestamp(timestamp)
# tempdf = pd.DataFrame(
#     {
#     'dates':dates,
#     'symbol':symbol,
#     'open':open_,
#     'close':close,
#     'high':high,
#     'low':low,
#     'volume':volume,
#     'adj_close':adj_close
#     }
# )
# tempdf

# Work on dataset

In [13]:
megadf = pd.DataFrame()

In [14]:
for jso in tqdm(symbol_list_dir):
    jsonf = pd.read_json(f'/kaggle/working/crypto_data/{jso}')
    jsondict = jsonf.to_dict()
    
    # Safely get the timestamp and check if it's None
    timestamp = jsondict.get('chart').get('result')[0].get('timestamp')
    if timestamp is None:
        print(f"Missing timestamp for {jso}. Skipping this symbol.")
        continue  # Skip this iteration if timestamp is None
    
    # Proceed with the rest of the code after ensuring timestamp is not None
    low = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('low')
    sym = jsondict.get('chart').get('result')[0].get('meta').get('symbol')
    
    # Create a list of symbols based on the length of timestamp
    symbol = list(sym for s in range(len(timestamp)))
    
    high = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('high')
    volume = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('volume')
    open_ = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('open')
    close = jsondict.get('chart').get('result')[0].get('indicators').get('quote')[0].get('close')
    adj_close = jsondict.get('chart').get('result')[0].get('indicators').get('adjclose')[0].get('adjclose')
    
    # Convert timestamp to dates
    dates = return_timestamp(timestamp)
    
    # Create a DataFrame
    tempdf = pd.DataFrame(
    {
        'dates': dates,
        'symbol': symbol,
        'open': open_,
        'close': close,
        'high': high,
        'low': low,
        'volume': volume,
        'adj_close': adj_close
    })
    
    # Concatenate the new data to the main DataFrame
    megadf = pd.concat([megadf, tempdf], ignore_index=True)


100%|██████████| 19/19 [00:00<00:00, 72.55it/s]

Missing timestamp for Nasdaq.json. Skipping this symbol.


In [15]:
megadf['symbol'].unique()

array(['DJT', 'FAST', 'BAC', 'CLSK', 'TLN', 'GOLD', 'AOS', 'SCHD', 'NVDA',
       'JPM', 'AFRM', 'UBER', 'CSAN', 'PLTR', 'MSTR', 'LYFT', 'TSLA',
       'ROCK'], dtype=object)

In [16]:
first_date = megadf['dates'][0]
last_date = megadf['dates'].iloc[-1]

In [17]:
first_date,last_date

('2021-09-30 13:30:00', '2024-10-11 16:25:42')

In [18]:
megadf.dtypes

dates         object
symbol        object
open         float64
close        float64
high         float64
low          float64
volume         int64
adj_close    float64
dtype: object

In [19]:
megadf['dates'] = pd.to_datetime(megadf['dates'], format="mixed")

In [20]:
megadf.dtypes

dates        datetime64[ns]
symbol               object
open                float64
close               float64
high                float64
low                 float64
volume                int64
adj_close           float64
dtype: object

In [21]:
megadf

,dates,symbol,open,close,high,low,volume,adj_close
0,2021-09-30 13:30:00,DJT,16.000000,9.950000,17.330000,9.900000,296100,9.950000
1,2021-10-01 13:30:00,DJT,10.000000,9.940000,11.000000,9.910000,153100,9.940000
2,2021-10-04 13:30:00,DJT,9.920000,9.920000,10.050000,9.900000,24300,9.920000
3,2021-10-05 13:30:00,DJT,10.020000,9.928000,10.020000,9.907000,8100,9.928000
4,2021-10-06 13:30:00,DJT,9.900000,9.920000,9.940000,9.840000,18600,9.920000
...,...,...,...,...,...,...,...,...
33325,2024-10-07 13:30:00,ROCK,70.790001,71.010002,71.279999,70.180000,103200,71.010002
33326,2024-10-08 13:30:00,ROCK,71.330002,70.489998,71.330002,70.000000,154300,70.489998
33327,2024-10-09 13:30:00,ROCK,70.669998,71.709999,71.989998,70.669998,114400,71.709999
33328,2024-10-10 13:30:00,ROCK,70.550003,70.160004,70.940002,69.260002,174500,70.160004


In [22]:
os.mkdir('dataset')

In [23]:
megadf.to_csv(f'dataset/1000_cryptos.csv',index = False)

# final dataset

In [24]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [25]:
df

,dates,symbol,open,close,high,low,volume,adj_close
0,2021-09-30 13:30:00,DJT,16.000000,9.950000,17.330000,9.900000,296100,9.950000
1,2021-10-01 13:30:00,DJT,10.000000,9.940000,11.000000,9.910000,153100,9.940000
2,2021-10-04 13:30:00,DJT,9.920000,9.920000,10.050000,9.900000,24300,9.920000
3,2021-10-05 13:30:00,DJT,10.020000,9.928000,10.020000,9.907000,8100,9.928000
4,2021-10-06 13:30:00,DJT,9.900000,9.920000,9.940000,9.840000,18600,9.920000
...,...,...,...,...,...,...,...,...
33325,2024-10-07 13:30:00,ROCK,70.790001,71.010002,71.279999,70.180000,103200,71.010002
33326,2024-10-08 13:30:00,ROCK,71.330002,70.489998,71.330002,70.000000,154300,70.489998
33327,2024-10-09 13:30:00,ROCK,70.669998,71.709999,71.989998,70.669998,114400,71.709999
33328,2024-10-10 13:30:00,ROCK,70.550003,70.160004,70.940002,69.260002,174500,70.160004


# Upload Dataset

# User secrets

In [26]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [27]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    js.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [28]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 3.84M/3.84M [00:02<00:00, 1.53MB/s]


Upload successful: 1000_cryptos.csv (4MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


In [29]:
shutil.rmtree('/kaggle/working/crypto_data')
shutil.rmtree('/kaggle/working/dataset')

# Thank you , and I will see you in next notebook :)